In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

/root/miniconda3/envs/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset["train"][1]

{'label': 1,
 'text': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."}

In [3]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [4]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(dataset["train"])

,label,text
0,3 stars,"If I have to choose a bar on Mill Ave. to goto, I goto Rula Bula. They've got a good selection of beer, food, and plenty of space to chill. I can look past the $2 weekend cover, because I guess compaired to some of the 10-25 dollar covers I've experienced while living in AZ, this ain't so bad. \n\nSometimes (forgot which days) they have nice live music here. There's a couple that comes in and one of them plays the fiddle really well (They have other non-irish performers as well). I still say they should get somekind of Irish/Celtic jukebox that continiously plays Irish drinking songs or something. I mean, it would be a big improvement on the random mix of hip-hop/techno they do play (when their radio is actually working). I guess asking for more Irish music in an \""Irish pub\"" is pretty silly of me."
1,3 stars,"Pros - Large Facility, multiple locations, Clean, Pool, saunas, showers, juice bar, classes, large variety of machines.\n\nCons - crowded parking, winter time shameless taking up of upfront parking spaces for \""valet parking\"" VALET PARKING at a gym?! Jersery shore/UFC wanna be's/ puffed up roid heads/ make up covered girls/ crowding and capitalizing equipment to \""hang out\"" and socialize. Staff is rude and have a condescending attitude towards us lesser folk. Trainers there tend to monopolize machines and space as they train 3 people at a time. I actually waited for 10 minutes to use a bench the trainer had planted all his \""stuff\"" with as he had one person do pushups next the the bench, moved yet another bench in front of it for another client to do step ups, and a third jumping rope on the other side. Apparently seeing that he was using the bench as just a bench as a fancy stop watch/clip board stand, I asked to use it. As I added the plates and lifted it off the rack the trainer had his client begin doing push ups again a foot away from me and RIGHT UNDER the plate! Where I quickly racked the bar and where he gave me a stern warning to be more careful of others and share space and equipment! \n\nGood gym, just ruined by rude staff and trainers who act like it's their kingdom and you are just annoying pest in their way."
2,2 star,This place has all the potential in the world to be super interesting but instead came off as a bit strange. We came here after reading all the rave reviews about breakfast. We sat down and waited a bit before any one approached and when they did it was not pleasant. Our server was barely interested in us and couldn't even bring herself to make eye contact with us. Our daughter who is quite the charmer couldn't even engage her. She was so disinterested that she brought us out the wrong drinks - not a good start. The food was completely and absolutely average. It was not worth the price or the rude service. \n\nWe will most likely not be back.
3,1 star,"I ordered service from LivingSocial to clean my apartment for a move out. I work 10 hour shifts and thought that the $90 for the service was a \""deal\"". I was soooo wrong!! I asked my mom to be at the empty apartment 15 mins before the schedule time and the cleaners NEVER showed. After an hour of having my mother waiting, I called the owner and he said that he didn't know what happened, but someone would be on their way shortly. Long story short, after waiting 3 hours, my mom left. I called the owner and he said that his cleaning crew got backed up on a previous job(is this my fault) and weren't able to make my appointment time. He then offered to reschedule, but when I called him back to set up a time all of my calls went to voicemail. This was almost two weeks ago and I haven't heard anything back yet. I've contacted LivingSocial for a refund and requested that this company be removed from the coupon deals."
4,3 stars,I only go here to watch ufc you will find good ufc fans here. Good beer
5,1 star,stewardess can I have another drink? Nothing more than airplane food in a nice place won't return
6,1 star,Not 

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [7]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,2 star,Long wait for small simple takeout order. Marinara sauce had lots of meat in it which is not good for vegetarians. Also forgot to include breadstick.,"[101, 3261, 3074, 1111, 1353, 3014, 1321, 3554, 1546, 119, 11391, 1611, 14313, 1125, 7424, 1104, 6092, 1107, 1122, 1134, 1110, 1136, 1363, 1111, 1396, 16609, 26559, 119, 2907, 9424, 1106, 1511, 8162, 23743, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


In [8]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [9]:
show_random_elements(train_dataset, num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,4 stars,"Had fun with my friends. We came here to watch Zedd perform, and he was amazing. The venue is nice. You can dance inside or chill outside by the pools. Fresh air at clubs is always a plus in my books.","[101, 6467, 4106, 1114, 1139, 2053, 119, 1284, 1338, 1303, 1106, 2824, 163, 23372, 3870, 117, 1105, 1119, 1108, 6929, 119, 1109, 6590, 1110, 3505, 119, 1192, 1169, 2842, 1656, 1137, 11824, 1796, 1118, 1103, 13546, 119, 15756, 1586, 1120, 3514, 1110, 1579, 170, 4882, 1107, 1139, 2146, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


In [10]:
show_random_elements(eval_dataset, num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,5 stars,"This is my favourite restaurant in Montreal: hands down, no contest, I want to marry it.\n I brought 3 restaurant-savvy New Yorkers with me for this food journey and they were blown away with how much Joe Beef rocks.\n\nIt was pouring buckets outside and the restaurant was warm, cozy and totally hopping when we arrived for our 7pm reservation. After standing there and trying to pick from all of the choices on the chalkboard, I finally hung my head and begged the server for her help. She laughed and then started listing her favourites. We ordered them all, everything from the duck, to the terrine to 3 types of fish. Every bite was incredible, tasty and knee-weakening delicious. The wine my dinner partner picked was perfect with all of our dishes, so good, that we ordered a second bottle.\n\nThe ambiance of this place is just plain awesome and I adore the buffalo in the washroom, he just makes me happy! This place is food porn at its best and I don't want help for my addiction.","[101, 1188, 1110, 1139, 9122, 4382, 1107, 5206, 131, 1493, 1205, 117, 1185, 5235, 117, 146, 1328, 1106, 5390, 1122, 119, 165, 183, 146, 1814, 124, 4382, 118, 21718, 1964, 7170, 1203, 20998, 1116, 1114, 1143, 1111, 1142, 2094, 5012, 1105, 1152, 1127, 10816, 1283, 1114, 1293, 1277, 2658, 16385, 2087, 5753, 119, 165, 183, 165, 183, 2240, 1204, 1108, 13587, 15398, 1116, 1796, 1105, 1103, 4382, 1108, 3258, 117, 1884, 6482, 1105, 5733, 6974, 2624, 1165, 1195, 2474, 1111, 1412, 128, 9952, 15702, 119, 1258, 2288, 1175, 1105, 1774, 1106, 3368, 1121, 1155, 1104, 1103, 9940, 1113, 1103, 23410, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

/root/miniconda3/envs/transformers/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/root/miniconda3/envs/transformers/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
!pip install --root-user-action=ignore  evaluate

DEPRECATION: Loading egg at /root/miniconda3/envs/transformers/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [16]:
!pip install  --root-user-action=ignore  scikit-learn

DEPRECATION: Loading egg at /root/miniconda3/envs/transformers/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [18]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [19]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
from transformers import TrainingArguments, Trainer

model_dir = "models/bert-base-cased-finetune-yelp"

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

In [21]:
print(training_args)

TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_always_pu

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [24]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

In [25]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.6104531288146973,
 'eval_accuracy': 0.24,
 'eval_runtime': 35.7699,
 'eval_samples_per_second': 2.796,
 'eval_steps_per_second': 0.363}

In [26]:
trainer.save_model(model_dir)

In [27]:
trainer.save_state()